In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import string, re
from bs4 import BeautifulSoup



In [ ]:
#Création des données
def lire_csv_vers_dictionnaire(chemin_fichier):
    # Lecture du fichier CSV avec pandas
    data = pd.read_csv(chemin_fichier, header=None)

    # Création du dictionnaire à partir des données
    dictionnaire = dict(zip(data.iloc[:, 0], data.iloc[:, 1]))

    return dictionnaire

# Exemple d'utilisation
chemin_fichier_solution = 'tblsolution.csv'
chemin_fichier_techno = 'tbltechno.csv'

dictionnaire_solution = lire_csv_vers_dictionnaire(chemin_fichier_solution)
dictionnaire_techno = lire_csv_vers_dictionnaire(chemin_fichier_techno)

print(len(dictionnaire_solution))
print(len(dictionnaire_techno))

1058
237


In [ ]:
# Chemin du fichier CSV
chemin_fichier = 'tbldictionnaire.csv'

# Charger les données CSV en tant que DataFrame pandas
data= pd.read_csv(chemin_fichier,header=None)


indices_a_supprimer = []




for index, row in data.iterrows():
    # Vérifier si la valeur à supprimer est présente dans la ligne
    if row[0] == "sol":
        if row[1] not in dictionnaire_solution.keys():
            indices_a_supprimer.append(index)

    elif row[0] == "tec":
        if row[1] not in dictionnaire_techno.keys():
            indices_a_supprimer.append(index)

# Supprimer les lignes collectées
data.drop(indices_a_supprimer, inplace=True)

# Récupérer le nombre de lignes
nbligne= data.shape[0]

In [ ]:
# Créer une nouvelle colonne vide
data[3] = ''  # Crée une colonne vide avec des chaînes vides
for i in range(nbligne):
  if data.iloc[i,0] == "tec":
    if data.iloc[i,1] in dictionnaire_techno:
      data.iloc[i,3] = dictionnaire_techno[data.iloc[i,1]]
  elif data.iloc[i,0]== "sol":
    if data.iloc[i,1] in dictionnaire_solution:
      data.iloc[i,3] = dictionnaire_solution[data.iloc[i,1]]

In [ ]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 37.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

from spacy.lang.fr.stop_words import STOP_WORDS as stopwords

# Charger le modèle spaCy en français

nlp = spacy.load("fr_core_news_sm")



# Fonction pour nettoyer une phrase

def clean_phrase(phrase):

    text_without_html = remove_html_tags(phrase)

    text_without_css = remove_css(text_without_html)

    doc = nlp(text_without_css)

    clean_words = ""

    for token in doc:

        if (token.text not in stopwords) and (token.lemma_ not in stopwords) and (token.pos_ != "PUNCT"):

            clean_words += f"{token.text.lower()} "

    return clean_words

# Fonction pour nettoyer les données dans un dataframe

def clean_data(dataframe):

    data_clean = pd.DataFrame()

    data_clean[0] = dataframe[0]

    data_clean[1] = dataframe[1]

    data_clean[2] = dataframe[2].apply(clean_phrase)

    data_clean[3] = dataframe[3]

    return data_clean
def remove_html_tags(text):
    # Utiliser BeautifulSoup pour supprimer les balises HTML
    soup = BeautifulSoup(text, "html.parser")
    cleaned_text = soup.get_text()
    return cleaned_text

def remove_css(text):
    # Utiliser une expression régulière pour supprimer les styles CSS
    cleaned_text = re.sub(r'<style[^>]*>[\s\S]*?</style>', '', text)
    return cleaned_text


In [ ]:
# Appliquer la fonction de nettoyage des données sur le dataframe
data_clean = clean_data(data)


<ipython-input-78-33201ced7877>:48: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [ ]:
display(data_clean)

,0,1,2,3
0,sol,2,installation frigorifique négative type cascad...,177
1,tec,2,management iso 50 001,1
2,sol,3,centrale négative mode booster,177
3,tec,3,gestion énergies,2
4,sol,4,arrêt compresseurs week end besoin refroidisse...,176
...,...,...,...,...
1290,sol,1710,favoriser communication digitale catalogue pap...,278
1291,sol,1711,remplacer chauffage bains gaz solution product...,266
1292,sol,1712,mettre place portes sectionnelles garder chale...,136
1293,sol,1713,films anti-chaleur baies vitrées,138


In [ ]:
# Charger le fichier CSV en utilisant des tabulations comme séparateurs et en spécifiant les noms des colonnes
df = data_clean
techMere = []

for index, row in df.iterrows():
    if (row[0] == "tec") and (row[3] == 1):
        techMere.append((row[2], row[1]))
    elif (row[0] == "tec") and (row[3] != 1):
        codeFils = row[3]
        for txt, code in techMere:
            if code == codeFils:
                df.loc[index, 2] = txt + " " + row[2]
                techMere.append((df.loc[index, 2], row[1]))

display(df)


,0,1,2,3
0,sol,2,installation frigorifique négative type cascad...,177
1,tec,2,management iso 50 001,1
2,sol,3,centrale négative mode booster,177
3,tec,3,management iso 50 001 gestion énergies,2
4,sol,4,arrêt compresseurs week end besoin refroidisse...,176
...,...,...,...,...
1290,sol,1710,favoriser communication digitale catalogue pap...,278
1291,sol,1711,remplacer chauffage bains gaz solution product...,266
1292,sol,1712,mettre place portes sectionnelles garder chale...,136
1293,sol,1713,films anti-chaleur baies vitrées,138


In [ ]:


# Remplacer les valeurs NaN par une chaîne vide
df[2] = df[2].fillna('')
nbSol = []
nbTec = []
# Utilisation de iterrows()
for index, row in df.iterrows():
    if row[0] == "sol":
      if row[1] not in nbSol:
        nbSol.append(row[1])
    elif row[0] == "tec":
      if row[1] not in nbTec:
        nbTec.append(row[1])
print(len(nbSol))
print(len(nbTec))



# Liste des codes et des textes
codes = df[1].tolist()  # Accéder à la quatrième colonne
texts = df[2].tolist()  # Accéder à la septième colonne
nature = df[0].tolist()
parent = df[3].tolist()


1058
237


In [ ]:
# Vectorisation des textes

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)


In [ ]:
def liste_solution(res,techno):
  for i in range(len(codes)):
    if nature[i] == "sol":
      if parent[i] == techno:
        if codes[i] not in res:
          res.append(codes[i])
    elif nature[i] == "tec":
      if parent[i] == techno:
        liste_solution(res,codes[i])

  return res


In [ ]:
def trouver_code(question):
    #Vectorisation de la question
    question_vec = vectorizer.transform([question])

    #Calcul des similarités cosinus entre la question et les textes de la base de données
    similarities = cosine_similarity(question_vec,X)

    #Trouver l'indice de la similarité maximale
    max_index = np.argmax(similarities)
    print(codes[max_index])

    if nature[max_index] == "sol":
      return codes[max_index]
    elif nature[max_index]=="tec":
      return liste_solution([],codes[max_index])


In [ ]:
# Exemple d'utilisation
question = input("Veuillez saisir votre question : ")
code_solution = trouver_code(question)

if isinstance(code_solution, list):
    print("Voici {} fiches à ce sujet :  ".format(len(code_solution)))
    for i in range(len(code_solution)):
      print("code K{}".format(code_solution[i]))
else:
    print("code K{}".format(code_solution))



Veuillez saisir votre question : Comment faire pour réduire la consommation de mon compresseur d'air comprimé ?
142
Voici 5 fiches à ce sujet :  
code K154
code K171
code K778
code K971
code K1092
